<a href="https://colab.research.google.com/github/codigoquant/python_para_investimentos/blob/master/08_An%C3%A1lise_de_Empresas_com_Python_no_Google_Colab_Python_para_Investimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ricos pelo Acaso

Link para o vídeo: https://youtu.be/e_ZRDG4F4ZA

# Importando Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

# Obtendo e tratando os dados - Web scraping

https://www.fundamentus.com.br/resultado.php

In [4]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [6]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [8]:
# pegando a primeira tabela, ajustando decimais, virgula por ponto
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [10]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
1,IVTT3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-0,40%",0.0,1.083050e+09,0.00,"20,67%"
2,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
3,MNSA4,0.47,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
4,CFLU4,1000.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"8,88%","10,72%",1.10,"17,68%","32,15%",0.0,6.035100e+07,0.06,"8,14%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,UBBR4,7.49,610.27,1.99,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
988,SRNA3,11.43,1161.34,1.31,1.550,"0,00%",0.344,15.05,6.05,-0.60,...,9.05,"25,62%","0,14%",1.16,"6,17%","0,11%",77336700.0,5.443630e+09,2.20,"35,93%"
989,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
990,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"


In [12]:
# substituindo virgula por ponto
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

# Analisando os dados - Fórmula Mágica

In [15]:
# filtrando empresas com liquidez de negociação diária, dos últimos dois meses
df = df[df['Liq.2meses'] > 1000000]

In [17]:
# melhores empresas, segundo a fórmula mágica.
# https://quantbrasil.com.br/magic-formula/
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
# valor da firma / geracao de xaixa operacional. Ver empresa que vale pouco em relacao à geração de caixa
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0 ].sort_values(by=['EV/EBIT'])['Papel'][:150].values
# retorno sobre o capital investido
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [19]:
# somar os indices, os menores valores são as melhores
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,PSSA3
1,2,WIZC3,PLPL3
2,3,GOAU4,ODPV3
3,4,PCAR3,TGMA3
4,5,QUAL3,LEVE3
...,...,...,...
145,146,VIVT3,AZZA3
146,147,SMFT3,ALOS3
147,148,TFCO4,ELET3
148,149,TOTS3,ELET6


In [21]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [23]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [25]:
t = pd.concat([a,b])
t

,ABEV3,AGRO3,ALOS3,ALUP11,AMOB3,ANIM3,ARML3,ASAI3,AURA33,AZUL4,...,ELMD3,EMBR3,ENEV3,FRAS3,ONCO3,ORVR3,PNVL3,RENT3,SRNA3,WEGE3
pos,117.0,90.0,54.0,57.0,126.0,20.0,59.0,13.0,70.0,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,26.0,77.0,147.0,85.0,NaN,58.0,49.0,27.0,10.0,16.0,...,48.0,131.0,140.0,68.0,136.0,115.0,137.0,111.0,142.0,9.0


In [27]:
# filtrando as que não estão nos dois rankings
rank = t.dropna(axis=1).sum()
rank

ABEV3     143.0
AGRO3     167.0
ALOS3     201.0
ALUP11    142.0
ANIM3      78.0
          ...  
VLID3      62.0
VTRU3      57.0
VULC3     130.0
WIZC3       8.0
YDUQ3     200.0
Length: 136, dtype: float64

In [29]:
# exclua seguradoras. Comparar com o site:
# https://quantbrasil.com.br/magic-formula/
rank.sort_values()[:15]

PSSA3     2.0
WIZC3     8.0
PLPL3    11.0
CMIN3    30.0
PETR4    31.0
PETR3    33.0
QUAL3    34.0
SYNE3    37.0
ASAI3    40.0
LEVE3    43.0
KEPL3    44.0
LREN3    47.0
VALE3    47.0
TGMA3    49.0
RECV3    49.0
dtype: float64

https://insight.economatica.com/existe-formula-magica-para-vencer-o-mercado

https://quantbrasil.com.br/magic-formula/